## Migrações com Alembic
***

Como discutimos anteriormente, precisaremos de utilizar o [Alembic](https://alembic.sqlalchemy.org/en/latest/) para criar tabelas e acompanhar as alterações na base de dados.

**Nota**: Este histórico de alterações é designado por migrações.

Para utilizar o Alembic, é necessário instalá-lo.

```sh
pip install alembic
alembic init alembic
```

Agora foram gerados muitos ficheiros no diretório do **alembic**. Para que o alembic se ligue à nossa base de dados precisamos de adicionar o seguinte ao ficheiro `env.py`.

Na verdade, não queremos armazenar nossas informações de conexão com o banco de dados no código, portanto, vamos usar variáveis de ambiente para armazená-las.

```py
...
import os

url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using
    username=os.environ.get('DB_USERNAME', 'notebook'),
    password=os.environ.get('DB_PASSWORD', 'notebook'),
    host=os.environ.get('DB_HOST', 'postgres'),
    database=os.environ.get('DB_NAME', 'notebook'),
    port=5432
)
config.set_main_option('sqlalchemy.url', str(url))
```

Com este código estamos a substituir o `sqlalchemy.url` do arquivo `alembic.ini` pelo url que criámos anteriormente.

Agora, para que o alembic veja as nossas tabelas, precisamos de atribuir `target_metadata` aos metadados da classe `Base`, importados do nosso arquivo principal para o arquivo `env.py`.

```py
...
from src.main.models import Base

...

target_metadata = Base.metadata
```

***
### Criando a migração
***

Para criar um novo script de migração, utilize o comando revision:

```sh
alembic revision -m "your message"
```

Substitua "sua mensagem" por uma breve descrição das mudanças que a migração irá fazer. Isto irá criar um novo arquivo no diretório `alembic/versões`.

`your_message_1fndmq03f.py` - o nome do arquivo de migração no diretório `alembic/versões`

***
### Editando uma migração
***

Abra o arquivo recém-criado e localize as funções `upgrade()` e `downgrade()`.

* `upgrade()` é aplicada quando a migração é executada

* `downgrade()` é aplicada quando uma migração é desfeita

Por exemplo, para adicionar uma nova tabela:

```py
def upgrade():
    op.create_table(
        'my_new_table',
        Column('id', Integer, primary_key=True),
        Column('name', String)
    )

def downgrade():
    op.drop_table('my_new_table')
```

***
### Rodando a migração
***

Para aplicar as migrações, rode:

```sh
alembic upgrade head
```

***
### Desfazer uma migração
***

Para desfazer a última migração, rode:

```sh
alembic downgrade -1
```

Para desfazer todas elas, rode:

```sh
alembic downgrade base
```

***
### Visualizar o status da ultima migração
***

Para ver o status da migração atual, rode:

```sh
alembic current
```

***
### Mostrar o histórico de migrações
***

Para ver todas as migrações e o seu estado (atualizado ou não), execute:

```sh
alembic history
```

Esta é uma introdução muito básica ao CLI do Alambique. Para uma utilização mais detalhada, por favor consulte a documentação oficial dele.

***
### Gerando migrações automáticas
***

O Alambique suporta a geração automática de scripts de migração a partir do estado atual dos seus modelos SQLAlchemy. Isso é feito usando a opção `--autogenerate` no comando `revision`. Veja como usá-la:

Antes de criar um script de migração automática, certifique-se de que o estado atual dos seus modelos SQLAlchemy corresponde ao estado do banco de dados. O Alembic irá comparar o estado atual dos seus modelos com o estado atual da sua base de dados e gerar um script de migração para reconciliar as diferenças.

Para gerar um script de migração automática, use o seguinte comando:

```sh
alembic revision --autogenerate -m "description of changes"
```

Por exemplo, se adicionou um modelo `User` aos seus modelos SQLAlchemy, o seu comando seria o seguinte:

```sh
alembic revision --autogenerate -m "added user model"
```

Isto irá criar um novo arquivo de script de migração no diretório `alembic/versions`.

***
### Vamos analisar o script gerado
***

A função de geração automática do Alembic não é perfeita e não capta todas as alterações que faz aos seus modelos. Portanto, é crucial verificar o script gerado e certificar-se de que ele corresponde às alterações que você fez nos seus modelos.

Abra o novo arquivo de script e verifique se as funções `upgrade()` e `downgrade()` refletem as alterações que fez aos seus modelos.

In [ ]:
"""initial migration
Revision ID: dcca43073c21
Revises:
Create Date: 2023-05-08 18:15:04.835447

"""
from alembic import op
import sqlalchemy as sa
from sqlalchemy.dialects import postgresql

# revision identifiers, used by Alembic.
revision = 'dcca43073c21'
down_revision = None
branch_labels = None
depends_on = None


def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.create_table('products',
        sa.Column('product_id', sa.INTEGER(), nullable=False),
        sa.Column('title', sa.String(length=255), nullable=False),
        sa.Column('description', sa.String(), nullable=False),
        sa.Column('created_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
        sa.Column('updated_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
        sa.PrimaryKeyConstraint('product_id')
    )
    op.create_table('users',
        sa.Column('telegram_id', sa.BIGINT(), nullable=False),
        sa.Column('full_name', sa.String(length=255), nullable=False),
        sa.Column('username', sa.String(length=255), nullable=True),
        sa.Column('language_code', sa.String(length=255), nullable=False),
        sa.Column('referrer_id', sa.BIGINT(), nullable=False),
        sa.Column('created_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
        sa.Column('updated_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
        sa.ForeignKeyConstraint(['referrer_id'], ['users.telegram_id'], ondelete='CASCADE'),
        sa.ForeignKeyConstraint(['referrer_id'], ['users.telegram_id'], ondelete='CASCADE'),
        sa.PrimaryKeyConstraint('telegram_id')
    )
    op.create_table('orders',
        sa.Column('order_id', sa.INTEGER(), nullable=False),
        sa.Column('user_id', sa.BIGINT(), nullable=False),
        sa.Column('created_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
        sa.Column('updated_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
        sa.ForeignKeyConstraint(['user_id'], ['users.telegram_id'], ondelete='CASCADE'),
        sa.ForeignKeyConstraint(['user_id'], ['users.telegram_id'], ondelete='CASCADE'),
        sa.PrimaryKeyConstraint('order_id')
    )
    op.create_table('order_products',
        sa.Column('order_id', sa.INTEGER(), nullable=False),
        sa.Column('product_id', sa.INTEGER(), nullable=False),
        sa.Column('quantity', sa.Integer(), nullable=False),
        sa.ForeignKeyConstraint(['order_id'], ['orders.order_id'], ondelete='CASCADE'),
        sa.ForeignKeyConstraint(['product_id'], ['products.product_id'], ondelete='RESTRICT'),
        sa.PrimaryKeyConstraint('order_id', 'product_id')
    )
    # ### end Alembic commands ###


def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_table('order_products')
    op.drop_table('orders')
    op.drop_table('users')
    op.drop_table('products')
    # ### end Alembic commands ###

Assim, sempre que quiser atualizar a sua base de dados, basta executar o comando `alembic upgrade head`.

Com isso cada modificação nas modelos criadas pode ser gerado uma nova migração para o banco de dados de modo controlado.